In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import country_converter as coco

import plotly.offline

In [2]:
path = 'data/yearly_full_release_long_format.csv'

In [98]:
data = pd.read_csv(path)

In [99]:
data

,Area,Country code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
0,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Clean,GW,0.19,NaN,NaN
1,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Fossil,GW,0.03,NaN,NaN
2,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Gas and Other Fossil,GW,0.03,NaN,NaN
3,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",GW,0.19,NaN,NaN
4,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Renewables,GW,0.19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316535,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Fossil,mtCO2,0.03,0.00,0.00
316536,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Renewables,mtCO2,0.00,0.00,NaN
316537,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Solar,mtCO2,0.00,0.00,NaN
316538,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Wind,mtCO2,0.00,0.00,NaN


In [180]:
data_clean = \
data[
#     (data['Year']==2021) & \
     (data['Category']=='Electricity generation') & \
     (data['Unit']=='%') & \
     (data['Subcategory']=='Fuel') & \
     (data['Area type']=='Country')][['Year','Area','Variable','Value']].set_index(['Year','Area','Variable']).unstack().droplevel(0,axis=1).reset_index()

In [181]:
GHG = {'Bioenergy':200,
       'Coal':1050,
       'Gas':450,
       'Hydro':10,
       'Nuclear':10,
       'Other Fossil':700,
       'Other Renewables':30,
       'Solar':40,
       'Wind':12}

In [182]:
data_clean

Variable,Year,Area,Bioenergy,Coal,Gas,Hydro,Nuclear,Other Fossil,Other Renewables,Solar,Wind
0,2000,Afghanistan,0.00,0.00,0.00,65.96,0.00,34.04,0.00,0.00,0.00
1,2000,Albania,0.00,0.00,0.00,96.23,0.00,3.77,0.00,0.00,0.00
2,2000,Algeria,0.00,0.00,95.48,0.21,0.00,4.31,0.00,0.00,0.00
3,2000,American Samoa,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00
4,2000,Angola,0.00,0.00,0.00,64.29,0.00,35.71,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
4568,2021,Ukraine,0.49,23.22,6.27,6.77,55.46,1.08,0.00,3.84,2.87
4569,2021,United Kingdom,12.73,1.93,40.23,1.82,15.26,2.79,0.00,4.06,21.17
4570,2021,United States of America,1.33,21.62,37.89,6.14,18.72,0.84,0.39,3.94,9.14
4571,2021,Uruguay,6.34,0.00,21.98,33.75,0.00,2.49,0.00,3.14,32.31


In [184]:
short_names = coco.convert(data_clean['Area'], to='ISO2', not_found='CG')
data_clean['Area short'] = short_names

Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex
Congo (the) not found in regex


In [186]:
data_clean.set_index(['Year','Area','Area short'], inplace=True)

In [187]:
data_clean['GHG intensity'] = (data_clean * GHG / 100).sum(1)

In [188]:
data_clean['Production'] = data[
#                                 (data['Year']==2021) & \
                                 (data['Category']=='Electricity generation') & \
                                 (data['Unit']=='TWh') & \
                                 (data['Subcategory']=='Fuel')].groupby(['Year','Area']).sum()['Value']

In [189]:
agg_c = {'Bioenergy':'Renewable',
         'Coal':'Fossil',
         'Gas':'Fossil',
         'Hydro':'Renewable',
         'Nuclear':'Nuclear',
         'Other Fossil':'Fossil',
         'Other Renewables': 'Renewable',
         'Solar': 'Renewable',
         'Wind':'Renewable',
         'GHG intensity':'GHG intensity',
         'Production':'Production'}

In [190]:
data_to_plot = (data_clean.groupby(agg_c, axis=1).sum()/[100,1,100,1,100]).reset_index()

In [196]:
fig = px.scatter_ternary(data_to_plot,
                         a='Fossil',
                         b='Nuclear',
                         c='Renewable',
                         hover_name='Area',
                         color='GHG intensity',
                         size='Production',
                         size_max=100,
                         height=1000,
                         animation_frame='Year',
#                          animation_group='Year',
                         text='Area short',
    color_continuous_scale='fall')

fig.update_layout(
    hovermode="closest",
    hoverlabel_namelength=-1,
#     height=200 + 250 * (i + 1),
    showlegend=False,
    margin=dict(l=50, r=10, b=200, t=100, pad=100),
#     font=C.figure_font,
    xaxis={'automargin': True},
    yaxis={'automargin': True}
)
fig.show()

In [199]:
plotly.offline.iplot(fig)